In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


**Preprocessing**

In [ ]:
images = []
labels = []

skin_types_dir = '/content/drive/MyDrive/skin2'
skin_types = os.listdir(skin_types_dir)

for skin_type in skin_types:
    folder_path = os.path.join(skin_types_dir, skin_type)
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        if os.path.isfile(img_path):

          try:

              if img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                  img = Image.open(img_path)
                  img = img.resize((224, 224))
                  img_array = np.array(img)
                  images.append(img_array)
                  labels.append(skin_types.index(skin_type))
              else:
                  print(f"Skipping non-image file: {img_path}")

          except Exception as e:
              print(f"Error loading image: {img_path}, Error: {e}")


images = np.array(images)
labels = np.array(labels)

images = images / 255.0

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.2, random_state=42
)

In [ ]:
X_train.shape

(1384, 224, 224, 3)

**Model Development**

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(skin_types), activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32,
                    validation_data=(X_test, y_test))

Epoch 1/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 128s 3s/step - accuracy: 0.2634 - loss: 4.1453 - val_accuracy: 0.4035 - val_loss: 1.2709
Epoch 2/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 140s 3s/step - accuracy: 0.4499 - loss: 1.2304 - val_accuracy: 0.4813 - val_loss: 1.1658
Epoch 3/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 0.5963 - loss: 1.0043 - val_accuracy: 0.5764 - val_loss: 1.0664
Epoch 4/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 127s 3s/step - accuracy: 0.7441 - loss: 0.7204 - val_accuracy: 0.6311 - val_loss: 1.1356
Epoch 5/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 141s 3s/step - accuracy: 0.8180 - loss: 0.5432 - val_accuracy: 0.6254 - val_loss: 1.0717
Epoch 6/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 146s 3s/step - accuracy: 0.8793 - loss: 0.3562 - val_accuracy: 0.6340 - val_loss: 1.2743
Epoch 7/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 128s 3s/step - accuracy: 0.9131 - loss: 0.2584 - val_accuracy: 0.6427 - val_loss: 1.2429
Epoch 8/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 129s 3s/step - accuracy: 0.9296 - loss: 0.2394 - val_accuracy: 0.6340 - v

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

11/11 ━━━━━━━━━━━━━━━━━━━━ 8s 646ms/step - accuracy: 0.7079 - loss: 1.1474
Test Loss: 1.3256
Test Accuracy: 0.6715
